**语言模型练习场01：**ColossalAI（SFT部分）
- https://github.com/hpcaitech/ColossalAI

**注意：**
- 此notebook**只演示在kaggle notebook下如何跑通ColossalAI的SFT部分**，并不会包含超参数的调整、对结果的分析等
- 类似的操作放到google colab理论上应该也可以跑通
- **如果你有自己的机器**，则此notebook对你的帮助可能不大（因为你不需要在notebook上进行训练）
- 此notebook的受众是**手里没有GPU资源，但是又想熟悉和浅浅尝试ColossalAI的小伙伴**

**数据的准备：**
1. 根据[官方文档的提示](https://github.com/hpcaitech/ColossalAI/tree/main/applications/Chat/examples)，在运行前需要准备好数据
2. 数据可以在[这里下载](https://github.com/XueFuzhao/InstructionWild/tree/main/data)。注意不要下载seed文件（因为seed文件只有instruction，而没有response），要下载README里面提到的json文件，例如instinwild_ch.json
3.将数据上传到Kaggle的Dataset中（需要创建自己的Dataset，可以为仅自己可见），按照Kaggle的步骤操作即可

**Kaggle Notebook的准备：**
1. 在界面右方**添加Dataset**，选择自己创建的Dataset。选择后，在代码中可以通过绝对路径访问。比如刚才创建的数据集名字叫做“instructdata”，其中我们上传了文件“instinwild_ch_small.json”，则在代码中，我们就可以通过这个路径访问数据集：/kaggle/input/instructdata/instinwild_ch_small.json
2. 最好选择**GPU T4x2**（Kaggle Notebook界面右方的Accelerator中选择）。如果选择P100可能会在安装过程中报错

**特别要注意的地方：**
1. 每周有**30个小时**的GPU使用时间
2. 每一次启动notebook**最长只能运行12个小时**（如果启动了但是不怎么使用，比如没有运行任何cell，也没有什么编辑的动作，可能也会在12个小时以内被强行终止。与google colab不同的时，长时间运行cell是可以的）
3. 一旦被终止，则**不能再找回输出的数据**！（输出的数据会放到/kaggle/working/路径下，如果需要里面的数据，必须在终止运行之前就下载下来）

## 安装环境

大体上是按照官方文档说明安装。**但是**，如果严格按照文档安装会报错。原因是ColossalAI是一个非常活跃的项目，每日都会有不同的代码变化。文档的部分内容可能还不能及时得到更新。所以，我们针对目前的情况，对安装顺序和细节做了一点微调。 **（当前时间为：2023年4月30日）**

相信在不远的将来，ColossalAI团队会把这些小bug修好，并且把文档立刻完善起来。

### 1 安装ColossalAI

执行完下面的命令后，此时你会发现，下载的文件是放在了/kaggle/working/ColossalAI下

In [1]:
!git clone https://github.com/hpcaitech/ColossalAI.git

Cloning into 'ColossalAI'...
remote: Enumerating objects: 24949, done.
remote: Counting objects: 100% (2362/2362), done.
remote: Compressing objects: 100% (479/479), done.
remote: Total 24949 (delta 1987), reused 2083 (delta 1881), pack-reused 22587
Receiving objects: 100% (24949/24949), 23.09 MiB | 29.19 MiB/s, done.
Resolving deltas: 100% (16582/16582), done.


**安装ColossalAI**

如果不执行这个安装，可能会出现这个[错误](https://github.com/hpcaitech/ColossalAI/issues/3629)：“ImportError: cannot import name 'ColoInitContext' from 'colossalai.zero'”

In [2]:
import os
os.chdir('./ColossalAI')
!pip install .

Processing /kaggle/working/ColossalAI
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.9/201.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 42.9 MB/s eta 0:00:00
  Created wheel for colossalai: filename=colossalai-0.2.8-py3-none-any.whl size=1059097 sha256=50ed72a86bf2ae29440764d5c46f67d000fcafbe8d4a5d8f6a947f5e57e85c70
  Stored in directory: /tmp/pip-ephem-wheel-cache-3th5gmed/wheels/3e/97

**安装transformers**

这里我们安装的是hpcaitech下的transformers，如果直接pip install transformers是否可行并没有测试

In [3]:
!git clone https://github.com/hpcaitech/transformers
os.chdir('./transformers')
!pip install .

Cloning into 'transformers'...
remote: Enumerating objects: 124468, done.
remote: Total 124468 (delta 0), reused 0 (delta 0), pack-reused 124468
Receiving objects: 100% (124468/124468), 127.08 MiB | 26.91 MiB/s, done.
Resolving deltas: 100% (93344/93344), done.
Processing /kaggle/working/ColossalAI/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=6790611 sha256=4220935232e4fb5bbdd639242eec8975f925c105da87c0d4d0137e013c5479a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-u7rm17k7/wheels/f8/7e/62/d660e4bfe297957f2a56ddb6284d5815eba12ca9dfe5b1cf73
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.4
    Uninstalling transformers-4.27.4:
      Successfully uninstalled transformers-4.27.4


**安装Chat部分需要的库**

In [4]:
os.chdir('/kaggle/working/ColossalAI/applications/Chat/')
!pip install .

Processing /kaggle/working/ColossalAI/applications/Chat
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 4.0 MB/s eta 0:00:00
  Created wheel for coati: filename=coati-1.0.0-py3-none-any.whl size=73195 sha256=114e624d66aa7c22966144210804c68036dda419db454e1fb5a6b657d58b5879
  Stored in directory: /tmp/pip-ephem-wheel-cache-pshxknq0/wheels/19/ab/40/58b2528cfb9dab45fa2cdceeff3538d85c2c72d65872c4de6a
Successfully built coati


我们还需要将**analyzer**的部分复制到当前操作系统的对应的**python packages**中（否则可能将来会报错：找不到analyzer）

如果你问，我怎么知道packages的路径是哪里呢？其实在上面执行各种pip install的过程中，就可以发现这个路径。

In [5]:
!cp -r /kaggle/working/ColossalAI/colossalai/_analyzer/ /opt/conda/lib/python3.7/site-packages/colossalai/

### 2 预训练模型的下载（以bloom为例）

首先**需要执行这个命令**（如果你是在colab中做尝试，可能也需要）

如果不安装下面的命令会出现什么情况呢？
- 从huggingface中git clone下来的模型看似下载下来了，但是其实下载下来的并不是实质的模型文件（如果你检查文件的大小，只有几B）
- 一旦下载下来的文件并不是实质的模型，则在运行SFT代码的时候会报错：**safetensors_rust.SafetensorError: Error while deserializing header: HeaderTooLarge**

In [6]:
!sudo apt-get install git-lfs
!git lfs install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 76 not upgraded.
Updated git hooks.
Git LFS initialized.


**下载ColossalAI支持的系列模型**，我们以bloomz-560m为例。在下面，我们将模型放在了/kaggle/working/中，但是这里并不是强制的，可以根据自己的喜欢变换位置。

In [7]:
os.chdir('/kaggle/working/')
!git clone https://huggingface.co/bigscience/bloomz-560m

Cloning into 'bloomz-560m'...
remote: Enumerating objects: 1332, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1332 (delta 3), reused 0 (delta 0), pack-reused 1322
Receiving objects: 100% (1332/1332), 7.18 MiB | 22.55 MiB/s, done.
Resolving deltas: 100% (616/616), done.
Filtering content: 100% (8/8), 2.11 GiB | 88.79 MiB/s, done.


### 3 运行SFT

In [8]:
os.chdir('/kaggle/working/ColossalAI/applications/Chat/examples')

**执行SFT代码**

我们这里是直接运行的py文件，如果你按照[文档的说明](https://github.com/hpcaitech/ColossalAI/tree/main/applications/Chat/examples)，运行sh脚本文件（!bash train_sft.sh）也是可以的。其本质上，都是运行这个train_sft.py文件。

在下面的命令中，为了演示
- 我们只是用一个非常非常非常小的数据集（--dataset）去跑程序（小到就只有5条数据）
- model：改为了“bloom”
- pretrain：改成了我们自己下载的模型地址
- save_path: 改成了我们想放的目录地址

**需要注意的是：**
- Kaggle Notebook的GPU是T4x2,所以显存大概有12+12=24G。所以我们在运行训练的时候可以设置**--nproc_per_node=2**（如果是1的话，则有一块GPU会闲置，显存也会砍半）
- 其他参数可能需要你自己去多多探索：比如lora、gradient checkingpoint等。更多参数的说明见[官方文档](https://github.com/hpcaitech/ColossalAI/tree/main/applications/Chat/examples)

In [9]:
!torchrun --standalone --nproc_per_node=2 train_sft.py \
    --pretrain "/kaggle/working/bloomz-560m" \
    --model 'bloom' \
    --strategy colossalai_zero2 \
    --log_interval 50 \
    --save_path  "/kaggle/working/bloomz-560m-finetuned" \
    --dataset "/kaggle/input/instructdata/instinwild_ch_small.json" \
    --batch_size 4 \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_datasets_size 512 \
    --max_epochs 1

[04/30/23 13:59:19] INFO     colossalai - colossalai - INFO:                    
                             /opt/conda/lib/python3.7/site-packages/colossalai/c
                             ontext/parallel_context.py:522 set_device          
                    INFO     colossalai - colossalai - INFO: process rank 0 is  
                             bound to device 0                                  
[04/30/23 13:59:24] INFO     colossalai - colossalai - INFO:                    
                             /opt/conda/lib/python3.7/site-packages/colossalai/c
                             ontext/parallel_context.py:558 set_seed            
                    INFO     colossalai - colossalai - INFO: initialized seed on
                             rank 0, numpy: 42, python random: 42,              
                             ParallelMode.DATA: 42, ParallelMode.TENSOR: 42,the 
                             default parallel seed is ParallelMode.DATA.        
                    INFO    

### 4 下载SFT完成的模型

In [10]:
os.chdir('/kaggle/working/')

理论上，我们是可以通过Kaggle Notebook右边的界面，选择想要下载的文件进行下载。但是由于Kaggle界面做的并不是很好，**经常会出现目录不能显示的问题**。好在，你应该是知道训练好的模型是放在了哪里。我们可以找到另一种办法将文件下载下来。

首先，我们把**完成的模型进行打包**（这样我们就可以一下字全都下载下来了，而不用一个一个的下载）

In [11]:
!tar -czvf bloomz-560m-finetuned.tar.gz bloomz-560m-finetuned

bloomz-560m-finetuned/
bloomz-560m-finetuned/config.json
bloomz-560m-finetuned/pytorch_model.bin
bloomz-560m-finetuned/tokenizer_config.json
bloomz-560m-finetuned/generation_config.json
bloomz-560m-finetuned/special_tokens_map.json
bloomz-560m-finetuned/tokenizer.json


完成打包后，我们**想办法获得下载链接**。执行下面的命令后，直接点击链接便可下载。

In [12]:
from IPython.display import FileLink
FileLink(r'bloomz-560m-finetuned.tar.gz')

/kaggle/working/bloomz-560m-finetuned.tar.gz

### 5 小结
仅仅跑起来只是一个开始。祝愿每个小伙伴都会最终获得更好的GPU资源、更棒的数据和更出色的属于自己的语言模型！